In [ ]:
!pip install ffmpeg

In [1]:
from transformers import pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from joblib import load
import numpy as np
import re

In [2]:
# Load Whisper model
pipe = pipeline(model="anggari/whisper-medium-multi", device='cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# Load LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(1, 3102), activation='relu'))
model.add(LayerNormalization()),
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(14, activation='softmax'))
model.load_weights('lstm_tfidf')

In [8]:
# Load vectorizer
tfidf_vectorizer = load('tfidf_vectorizer_new.joblib')

# Load label encoder
label_encoder = load('label_encoder.pkl')

In [4]:
import os
os.environ['HF_DATASETS_OFFLINE '] = "1"
from datasets import load_dataset, concatenate_datasets

In [5]:
lang_used=['zh-CN', 'ru-RU', 'fr-FR', 'en-US', 'de-DE']
minds=[]
for i in lang_used:
    minds_data = load_dataset("PolyAI/minds14", i, trust_remote_code=True)
    minds.append(minds_data['train'])

minds = concatenate_datasets(minds)
minds

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 2754
})

In [71]:
import random
random.randint(0,2754)

1649

In [6]:
sample = minds[1649]

In [7]:
sample

{'path': 'C:\\Users\\andre\\.cache\\huggingface\\datasets\\downloads\\extracted\\482a593b1882116f1ef934ce05f74608ebffded5fe81b88e49a3d63337574cf4\\en-US~BALANCE\\602ba046bb1e6d0fbce91fd3.wav',
 'audio': {'path': 'C:\\Users\\andre\\.cache\\huggingface\\datasets\\downloads\\extracted\\482a593b1882116f1ef934ce05f74608ebffded5fe81b88e49a3d63337574cf4\\en-US~BALANCE\\602ba046bb1e6d0fbce91fd3.wav',
  'array': array([ 0.        ,  0.        ,  0.00024414, ...,  0.00024414,
          0.        , -0.00024414]),
  'sampling_rate': 8000},
 'transcription': 'what is my current bank balance',
 'english_transcription': 'what is my current bank balance',
 'intent_class': 4,
 'lang_id': 4}

# Inference Pipeline

In [9]:
def remove_double_space(text):
    # Menghapus double space dengan regex
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text

def transcribe(audio):
    text = pipe(audio)["text"]
    text = remove_double_space(text).lower()
    text_tfidf = tfidf_vectorizer.transform([text])
    text_dense = text_tfidf.toarray()
    text_lstm = np.expand_dims(text_dense, axis=1)
    y_pred = model.predict(text_lstm)
    intent_class = np.argmax(y_pred, axis=1)
    intent = label_encoder.inverse_transform(intent_class)
    return text, intent_class, intent

In [10]:
transcribe(sample["audio"]["array"])

C:\Users\andre\python310\lib\site-packages\transformers\models\whisper\generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


1/1 [==============================] - 0s 371ms/step


('what is my current bank balance',
 array([4], dtype=int64),
 array(['balance'], dtype=object))